In [1]:
import os

import pandas as pd

# Customizing pandas output
pd.set_option("display.max_rows", 100, "display.max_columns", 25)

In [2]:
ac_pwr_df = pd.read_csv("../data/cleaned/ac_pwr_cleaned.csv",
                        parse_dates=["timestamp"]
                       )

display(ac_pwr_df.head())

,timestamp,ac_1,ac_2,ac_3,ac_4,ac_5,ac_6,ac_7,ac_8,ac_9,ac_10,ac_11,ac_12,ac_13,ac_14,ac_15,ac_16,ac_17,ac_18
0,2019-08-01 00:00:00,7.518632,8.788315,0.0,0.0,2.617045,4.079041,2.782276,4.624447,5.222060,2.151238,1.585072,0.560373,3.142941,2.749470,5.417774,4.113460,3.305072,6.735981
1,2019-08-01 00:01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-08-01 00:02:00,7.426114,8.940615,0.0,0.0,2.581625,3.781231,2.529366,5.057423,5.349465,2.414715,2.168184,1.818730,3.085110,2.720484,3.302422,3.986483,3.220588,6.379500
3,2019-08-01 00:03:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-08-01 00:04:00,7.052986,9.161103,0.0,0.0,2.592095,3.800127,2.332304,6.322521,3.995392,2.237114,3.345624,2.310409,3.132799,2.676861,3.539026,3.797881,3.131560,6.363475


In [3]:
ac_pwr_df.timestamp.min(), ac_pwr_df.timestamp.max()

(Timestamp('2019-08-01 00:00:00'), Timestamp('2019-09-30 23:59:00'))

In [4]:
ac_pwr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87840 entries, 0 to 87839
Data columns (total 19 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  87840 non-null  datetime64[ns]
 1   ac_1       43708 non-null  float64       
 2   ac_2       43711 non-null  float64       
 3   ac_3       43713 non-null  float64       
 4   ac_4       43704 non-null  float64       
 5   ac_5       43716 non-null  float64       
 6   ac_6       43717 non-null  float64       
 7   ac_7       43679 non-null  float64       
 8   ac_8       43704 non-null  float64       
 9   ac_9       43654 non-null  float64       
 10  ac_10      43690 non-null  float64       
 11  ac_11      43670 non-null  float64       
 12  ac_12      43713 non-null  float64       
 13  ac_13      43719 non-null  float64       
 14  ac_14      43696 non-null  float64       
 15  ac_15      43715 non-null  float64       
 16  ac_16      43710 non-null  float64      

### Background:

There are 18 ACs installed in one of the hotel located in [Gurgaon](https://en.wikipedia.org/wiki/Gurgaon). The sensors collects power consumption by ACs every minute. Using this available data will work on the following forecasting techniques:

* **Intraday:** forecasting the power consumed by the hotel in an hour


* **Day-ahead:** forecasting the power consumed by the hotel in a day


* **Long-term:** forecasting the power consumed by the hotel in a week


**Note:** The data is available from `2019-08-01 00:00:00` to `2019-09-30 23:59:00`

### To-do:

* Fill missing value by linear interpolation

* Resample the data

## Filling missing values by linear method of interpolation

In [5]:
for col in ac_pwr_df.columns[1:]:
    ac_pwr_df[col] = ac_pwr_df[col].interpolate(method="linear")

### Dumping the filled data

In [6]:
ac_pwr_df.to_csv("../data/cleaned/ac_pwr_na_filled.csv", index=False)

## Resampling the data

### 1. Hourly power consumption

In [7]:
ac_pwr_df_time_index = ac_pwr_df.set_index("timestamp")

ac_pwr_df_hourly_resampled = ac_pwr_df_time_index.resample("1H").mean()
ac_pwr_df_hourly_resampled["hotel"] = ac_pwr_df_hourly_resampled.sum(axis=1)

display(ac_pwr_df_hourly_resampled.head())

,ac_1,ac_2,ac_3,ac_4,ac_5,ac_6,ac_7,ac_8,ac_9,ac_10,ac_11,ac_12,ac_13,ac_14,ac_15,ac_16,ac_17,ac_18,hotel
timestamp,,,,,,,,,,,,,,,,,,,
2019-08-01 00:00:00,6.831317,8.499431,1.818682,0.975186,2.226705,4.305690,2.678105,4.827529,3.631382,2.561074,1.648186,2.027200,4.716227,5.231973,5.657984,4.252056,2.629953,5.148527,69.667207
2019-08-01 01:00:00,7.409921,8.464055,1.473305,1.842207,2.231021,4.304568,2.603761,4.467366,3.053147,2.660621,1.357206,1.893804,5.089799,5.689334,5.939549,4.606612,2.574381,3.971865,69.632524
2019-08-01 02:00:00,7.356081,8.112770,1.492203,1.478549,1.997557,4.232891,2.773172,3.792759,3.116219,2.803366,1.544293,1.912718,4.031990,5.029272,6.225177,5.284894,2.741124,3.949288,67.874325
2019-08-01 03:00:00,6.633610,7.908404,2.250147,1.452142,2.087573,4.725451,3.021327,3.897844,2.845015,2.729238,1.117579,1.677892,4.406184,5.016430,4.910233,4.990273,4.248621,3.898252,67.816213
2019-08-01 04:00:00,6.595406,7.297426,1.733085,2.556184,1.869750,3.606569,2.927028,3.757071,3.385324,2.599631,1.211025,1.831026,3.983865,4.278251,5.326393,4.815338,4.176402,3.938632,65.888407


### 2. Daily Power Consumption

In [8]:
ac_pwr_df_daily_resampled = ac_pwr_df_hourly_resampled.resample("1D").sum()

display(ac_pwr_df_daily_resampled.head())

,ac_1,ac_2,ac_3,ac_4,ac_5,ac_6,ac_7,ac_8,ac_9,ac_10,ac_11,ac_12,ac_13,ac_14,ac_15,ac_16,ac_17,ac_18,hotel
timestamp,,,,,,,,,,,,,,,,,,,
2019-08-01,103.470026,172.626486,54.381934,79.686773,34.608149,67.184130,62.564120,78.266052,54.186934,42.277052,43.144660,60.953799,113.237256,73.486446,103.163494,86.888814,111.446902,181.466293,1523.039318
2019-08-02,95.457144,119.934880,69.841698,75.753428,39.334874,56.177836,56.024626,35.606027,63.244424,42.419516,48.275655,67.084719,122.974600,56.484863,107.289684,82.610542,106.667588,186.896237,1432.078339
2019-08-03,78.115544,102.989347,70.306946,45.665104,32.989481,61.329650,76.354959,30.810134,66.082144,35.093054,52.161226,73.864985,115.368455,44.766982,107.147242,70.312478,105.233074,189.204615,1357.795419
2019-08-04,101.042185,106.826077,63.270812,37.968961,37.327546,86.757756,68.416550,38.187965,61.107829,41.541297,44.848667,73.931385,119.428521,53.830066,111.885421,70.253436,109.332319,201.149405,1427.106201
2019-08-05,84.887521,96.214918,42.860398,21.841656,40.154765,91.051414,59.637846,39.431319,62.324197,34.170599,80.340770,74.575773,100.270028,67.084092,120.327408,83.205266,91.530683,146.565396,1336.474049


### 3. Monthly power consumed

In [9]:
ac_pwr_df_monthly_resampled = ac_pwr_df_daily_resampled.resample("1M").sum()

display(ac_pwr_df_monthly_resampled)

,ac_1,ac_2,ac_3,ac_4,ac_5,ac_6,ac_7,ac_8,ac_9,ac_10,ac_11,ac_12,ac_13,ac_14,ac_15,ac_16,ac_17,ac_18,hotel
timestamp,,,,,,,,,,,,,,,,,,,
2019-08-31,1808.747705,3068.664994,1849.769082,1813.136503,1003.242472,1708.330609,2293.245050,1824.605130,2325.819118,1199.733407,1671.993978,1963.945054,3178.051217,2044.843764,3184.473209,2261.233459,3049.238938,4856.515850,41105.589537
2019-09-30,1670.159125,2585.140058,1905.585589,1689.534722,1076.005558,1975.884665,2065.828122,1901.942419,2327.528536,1049.225176,1849.183572,1984.228781,3079.595455,2221.426605,2731.124457,1833.696278,3124.047090,4601.191524,39671.327730


### 4. Final power consumed

In [10]:
ac_pwr_df_total_consumed = ac_pwr_df_monthly_resampled.iloc[:, :-1].sum().reset_index()
ac_pwr_df_total_consumed = ac_pwr_df_total_consumed.rename(columns={"index": "ac", 0: "power_consumed"})
ac_pwr_df_total_consumed = ac_pwr_df_total_consumed.sort_values(by="power_consumed").reset_index(drop=True)

display(ac_pwr_df_total_consumed)

,ac,power_consumed
0,ac_5,2079.248030
1,ac_10,2248.958583
2,ac_1,3478.906830
3,ac_4,3502.671225
4,ac_11,3521.177550
5,ac_6,3684.215274
6,ac_8,3726.547549
7,ac_3,3755.354671
8,ac_12,3948.173835
9,ac_16,4094.929737


### Dumping the resampled data

In [11]:
os.makedirs("../data/resampled/", exist_ok=True)

In [12]:
ac_pwr_df_hourly_resampled.to_csv("../data/resampled/ac_pwr_hourly.csv")
ac_pwr_df_daily_resampled.to_csv("../data/resampled/ac_pwr_daily.csv")
ac_pwr_df_monthly_resampled.to_csv("../data/resampled/ac_pwr_monthly.csv")
ac_pwr_df_total_consumed.to_csv("../data/resampled/ac_pwr_total_consumed.csv", index=False)